# Performance of models against layer point data 

In [ ]:
chunk_start

In [ ]:
matchup = ["surface", "all"] 

In [ ]:

model_dict = model_dict_str
num_models = len(model_dict)

In [ ]:
# now, figure out which variables are available
vars1 = [os.path.basename(x).split("_")[-1].replace(".csv", "") for x in glob.glob(list(model_dict.values())[0] + f"/oceanval_matchups/point/**/**/**/*.csv")]
vars1 = [x for x in vars1 if "unit" not in x and "paths" not in x]
vars2 = [os.path.basename(x).split("_")[-1].replace(".csv", "") for x in glob.glob(list(model_dict.values())[1] + f"/oceanval_matchups/point/**/**/**/*.csv")]
vars2 = [x for x in vars2 if "unit" not in x and "paths" not in x]
variables= [x for x in vars1 if x in vars2]

In [ ]:
n_show = 0
for vv in variables:

    i = 0
    i_subfigure = 1
    for key, value in model_dict.items():
        key_path = [x for x in glob.glob(value + f"/oceanval_matchups/point/**/{vv}/**/*.csv") if "unit" not in x and "paths" not in x][0]
        if i == 0:
            df1 = pd.read_csv(key_path)
            ff_def = key_path.replace(".csv", "_definitions.pkl")
            with open(ff_def, "rb") as f:
                definitions = pickle.load(f)
            short_name = definitions[vv].short_name
            short_title = definitions[vv].short_title 
            name1 = os.path.basename(key_path).split("_")[0]
            if "depth" in df1.columns:
                df1 = df1.query("depth < 5") 
                # ditch depth
                df1 = df1.drop(columns = "depth")
            valid_columns = ["lon", "lat", "year", "month", "day"]
            valid_columns = [x for x in valid_columns if x in df1.columns]
            df1 = df1.groupby(valid_columns).mean().reset_index()
            df1 = df1.rename(columns = {"model": key})
        else:
            df2 = pd.read_csv(key_path)
            name2 = os.path.basename(key_path).split("_")[0]
            if "depth" in df2.columns:
                df2 = df2.query("depth < 5")
                df2 = df2.drop(columns = "depth")
            valid_columns = ["lon", "lat", "year", "month", "day"]
            valid_columns = [x for x in valid_columns if x in df2.columns]
            df2 = df2.groupby(valid_columns).mean().reset_index()
            df2 = df2.rename(columns = {"model": key})
            df = pd.merge(df1, df2)
        i+=1
        
    total = len(df)

    if len(df) == 0:
        continue
    md(f"## Performance of {short_name}")
        # melt baseline, predators
    valid_columns.append("observation")
    df = pd.melt(df, id_vars = valid_columns, var_name = "model", value_name = "value") 

    # calculate the r2 for both models
    # import r2_score
    r2 = df.groupby("model").apply(lambda x: np.corrcoef (x["observation"], x["value"])[0,1]).reset_index() 
    # turn this into a dictionary
    r2 = r2.set_index("model").to_dict()[0]
    # round the r2 values to 3 decimal places
    r2 = {k:round(v, 3) for k,v in r2.items()}
    keys = list(r2.keys())

    # This should be read in using package data
    data_path = str(importlib.resources.files("oceanval").joinpath("data/TM_WORLD_BORDERS-0.3.shp"))

    shape = gpd.read_file(data_path)
    xlim = [df.lon.min(), df.lon.max()]
    ylim = [df.lat.min(), df.lat.max()] 


    md(f"A total of {total} matchups were found for {short_name}. The locations of these matchups are shown in the figure below. The x-axis represents the longitude, while the y-axis represents the latitude. The grey background represents the world map.")
    if name1 == name2:
        md(f"The matchups were with observational data from **{name1.upper()}**.")

    gg = (
        ggplot(df)+
         geom_point(aes(x  = "lon",y =   "lat"))+
        geom_map(shape, aes("LON", "LAT"), fill = "grey", colour = "grey")+
        coord_cartesian(xlim = xlim, ylim = ylim)+
        #scale_x_continuous(breaks = [-20, -10, 0, 10], labels = ["20°W", "10°W", "0°", "10°E"])+
        #scale_y_continuous(breaks = [40, 50, 60], labels = ["40°N", "50°N", "60°N"])+
        theme_bw(base_size = 10)+
        theme(axis_title_x=element_blank(),
                axis_title_y=element_blank())
    )
    gg.draw()

    md(f"**Figure {i_figure}.{i_subfigure}**. Locations of matchups for {short_name}.")
    i_subfigure+=1
    
    gg = (
        ggplot(df,aes(x = "value", y = "observation")) +
        geom_point()+
        geom_smooth(method = "lm", colour = "blue")+
        geom_abline(intercept = 0, slope = 1, colour = "red")+
        facet_wrap("~model", scales = "free")+
        labs(x = f"{key} simulation {short_name}", y = f"Observed {short_name}") 
    
    )

    gg.draw()

    md(f"**Figure {i_figure}.{i_subfigure}**. Simulated vs observed {short_name}. The red line represents a 1:1 relationship, while the blue line represents the linear regression fit.")
    i_subfigure+=1
    md(f"The **correlation coefficient** between the model and observations for the simulations are as follows: {r2[keys[0]]} for {keys[0]} and {r2[keys[1]]} for {keys[1]}.")

    df_comp = df1.merge(df2)
    x_var = [x for x in df_comp.columns if x not in valid_columns][0]
    y_var = [x for x in df_comp.columns if x not in valid_columns][1]
    gg = (
        ggplot(df_comp, aes(x = x_var, y = y_var))+
        geom_point()+
        labs(x = f"{x_var} simulation {short_name}", y = f"{y_var} {short_name}")+
        geom_smooth(method = "lm", colour = "blue")+
        geom_abline(intercept = 0, slope = 1, colour = "red")
    )

    gg.draw()

    md(f"**Figure {i_figure}.{i_subfigure}**. Comparison of {x_var} and {y_var} for {short_name} between models. The red line represents a 1:1 relationship, while the blue line represents the linear regression fit.")
    i_subfigure+=1

    i_figure+=1
    n_show+=1

if n_show == 0:
    md("No matchups were available in both simulations.")